### Тестовое задание на стажировку в ВК

Подготовка и проверка датасета.

In [1]:
import pandas as pd
import numpy as np
import sweetviz as sv

from catboost import CatBoostRanker, Pool, MetricVisualizer
import xgboost as xgb
from sklearn.metrics import ndcg_score
from sklearn.metrics import top_k_accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [55]:
df = pd.read_csv('data/intern_task.csv')

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 

In [4]:
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns') 

In [5]:
df.head(100)

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.000000,0.333333,0.000000,1.000000,10.0,0.0,9.0,11.0,30.0,8.198216,22.240604,18.240926,21.439961,8.194138,0.000000,0.0,1.0,5.0,7.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,4.0,0.000000,0.000000,0.333333,1.666667,2.333333,0.222222,0.000000,0.222222,0.222222,1.555556,0.100000,0.000000,0.111111,4.822520,0.233333,0.000000,0.000000,0.000000,0.090909,0.033333,0.100000,0.000000,0.111111,0.181818,0.133333,0.033333,0.000000,0.037037,0.151515,0,0,0.000000,0.002743,0.001837,0.001728,5.183394,0.000000,1,36.563118,24.430262,0.000000,0.00000,0.000000,6.316804,1.375925,5.183394,0.000000,8.655534,19.334808,20.730882,1.727798,0.000000,2.885178,12.187706,8.143421,5.970573,0.000000,16.648505,29.059080,79.371724,0.0,0.0,0.0,1.0,1.0,0,0.000000,0.786755,0.971298,0.946833,11.904045,0.000000,7.738967,26.350077,22.927740,-19.201910,-20.839029,-17.627562,-7.865720,-9.926361,-22.574108,-24.810069,-21.077948,-20.642055,-19.589214,-21.995388,-23.636237,-20.421283,-6.122882,-8.437561,3.0,76.0,0.0,0.0,153.0,3866.0,17.0,104.0,0.0,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.000000,1.000000,0.004251,1.000000,557.0,0.0,7.0,11.0,575.0,8.198216,22.240604,18.240926,21.439961,8.194138,0.000004,0.0,4.0,0.0,48.0,6.0,0.0,1.0,0.0,7.0,31.0,0.0,2.0,0.0,33.0,8.501341,0.000000,1.333333,0.000000,16.000000,133.555556,0.000000,0.222222,0.000000,144.666667,0.078995,0.000000,0.571429,3.445965,0.083478,0.010772,0.000000,0.142857,0.000000,0.012174,0.055655,0.000000,0.285714,0.000000,0.057391,0.026332,0.000000,0.190476,0.000000,0,0,0.000000,0.004535,0.000000,0.000438,112.486713,0.000000,1,0.000000,122.951509,4.141566,0.00000,4.695805,0.000000,4.815739,72.061386,0.000000,9.779175,0.000000,76.674007,37.495571,0.000000,7.710171,0.000000,40.983836,769.584565,0.000000,4.753632,0.000000,860.715989,1.0,0.0,1.0,0.0,1.0,0,0.000000,0.948870,0.000000,0.792060,23.900332,0.000000,21.651734,0.000000,24.339780,-12.018080,-20.839029,-7.961910,-24.026420,-11.733085,-16.087395,-24.810069,-20.081042,-27.079631,-15.792263,-12.102374,-23.636237,-5.459244,-26.827529,-11.848628,3.0,73.0,0.0,9.0,266.0,56137.0,5.0,2.0,0.0,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.000000,0.666667,0.008501,1.000000,522.0,0.0,6.0,8.0,536.0,8.

In [6]:
df.shape

(235258, 146)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
rank,235258.0,6.778686e-01,8.300334e-01,0.000000,0.000000,0.000000,1.000000e+00,4.000000e+00
query_id,235258.0,1.482841e+04,8.193945e+03,10.000000,8215.000000,14935.000000,2.158000e+04,2.999500e+04
feature_0,235258.0,1.911960e+00,1.237374e+00,0.000000,1.000000,2.000000,3.000000e+00,3.100000e+01
feature_1,235258.0,2.062332e-01,5.790888e-01,0.000000,0.000000,0.000000,0.000000e+00,1.800000e+01
feature_2,235258.0,1.189847e+00,1.037233e+00,0.000000,0.000000,1.000000,2.000000e+00,2.700000e+01
feature_3,235258.0,5.502725e-01,7.909465e-01,0.000000,0.000000,0.000000,1.000000e+00,9.000000e+00
feature_4,235258.0,1.960082e+00,1.203534e+00,0.000000,1.000000,2.000000,3.000000e+00,3.100000e+01
feature_5,235258.0,8.039641e-01,3.399551e-01,0.000000,0.666667,1.000000,1.000000e+00,1.000000e+00
feature_6,235258.0,9.755747e-02,2.660348e-01,0.000000,0.000000,0.000000,0.000000e+00,1.000000e+00
feature_7,235258.0,5.411739e-01,4.199727e-01,0.000000,0.000000,0.500000,1.000000e+00,1.000000e+00


С целью ознакомления с датасетом проведем разведочный анализ.  
Для это воспользуемся готовой библиотекой sweetviz.

In [8]:
report = sv.analyze([df, "Permit Logs"])

                                             |                  | [  0%]   00:00 -> (? left)

C:\ProgramData\Anaconda3\lib\site-packages\sweetviz\graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dataframe[feature] = pd.Series(dtype=float)
C:\ProgramData\Anaconda3\lib\site-packages\sweetviz\graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dataframe[feature] = pd.Series(dtype=float)
C:\ProgramData\Anaconda3\lib\site-packages\sweetviz\graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

In [9]:
report.show_html('report.html')

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Полученные отчеты находятся в папке eda_reports.

In [10]:
from IPython.display import display, HTML, IFrame

In [11]:
IFrame(src = 'report.html', width = 1000, height = 500)

При изучении отчетов EDA, было выявлено 13 категориальных признаков, включая константы  
(feature_64, feature_65, feature_72, feature_100),  
бинарные (feature_95, feature_96, feature_97, feature_98, feature_99).  
Остальные признаки, определенные как категориальные, имеют до 10 значений.

In [64]:
const_cols_df = df.loc[:, df.nunique() == 1].columns.to_list()
const_cols_df

['feature_64', 'feature_65', 'feature_72', 'feature_100']

Удалим категориальные признаки

In [65]:
df.drop(const_cols_df, axis=1, inplace=True)

Составим матрицу корреляции Пирсона и рассмотрип признаки с наибольшей корреляцией

In [14]:
# df.corr()

In [15]:
def correl(X_train, thresh):
    cor = X_train.corr()
    c1 = cor.stack().sort_values(ascending=False).drop_duplicates()
    all_cor = c1[c1.values != 1]
    return all_cor[abs(all_cor) > thresh]

In [16]:
correl(df, 0.95)

feature_35   feature_8      1.000000
feature_20   feature_8      1.000000
feature_35   feature_20     1.000000
feature_19   feature_15     0.999892
feature_74   feature_70     0.999599
feature_90   feature_94     0.999323
feature_20   query_id       0.999009
feature_35   query_id       0.999009
query_id     feature_8      0.999009
feature_24   feature_142    0.999007
feature_123  feature_113    0.997952
feature_111  feature_121    0.997734
feature_119  feature_115    0.997582
feature_44   feature_40     0.997458
feature_34   feature_30     0.997357
feature_80   feature_84     0.997088
feature_10   feature_14     0.997014
feature_39   feature_143    0.996818
feature_85   feature_89     0.996235
feature_114  feature_124    0.996158
feature_29   feature_25     0.996002
feature_75   feature_79     0.995725
feature_120  feature_110    0.995705
feature_122  feature_112    0.994764
feature_16   feature_17     0.991783
feature_110  feature_114    0.991119
feature_120  feature_124    0.989141
f

Разделим датсет на тренировочную и тестовую выборку так, чтобы в выборку попали объекты с уникальным query_id.

In [66]:
df['query_id'].nunique()

2000

In [67]:
grouped = df.groupby('query_id').sum().reset_index()

In [68]:
grouped.iloc[1499,0], grouped.iloc[1500,0]

(22495, 22510)

In [69]:
df_train = df[df['query_id'] <= 22495]
df_test = df[df['query_id'] > 22495]

In [70]:
X_train = df_train.drop(["rank", "query_id"], axis=1)
y_train = df_train.iloc[:,0]
queries_train = df_train.iloc[:,1]

In [71]:
X_test = df_test.drop(["rank"], axis=1)
y_test = df_test.iloc[:,0]
queries_test = df_test.iloc[:,1]

Для расчета такие показатели, как NDCG и найденная релевантность, должны находиться в сегменте [0,1].

Статистика помечена значениями значимости

0 - неактуально, 1 - очень важно. В таблице представлено количество документов для каждого значения.

In [72]:
from collections import Counter
Counter(y_train).items()

dict_items([(0, 100588), (1, 57486), (2, 23561), (4, 1215), (3, 2938)])

In [73]:
max_relevance = np.max(y_train)
y_train = y_train / max_relevance
y_test = y_test / max_relevance

Создаим простую модель CatBoost

In [27]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [28]:
model_cb = CatBoostRanker(
    metric_period = 20,
    max_depth =  10,
    eval_metric = 'NDCG:top=5',
    random_seed = 42,
    task_type = "GPU"
)
model_cb.fit(
    train,
    eval_set = test,
    verbose=False
)

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [29]:
# NDCG@5 на тренировочной выборке

y_pred_train = model_cb.predict(X_train)
ndcg_score([y_train], [y_pred_train], k=5)

0.9999999999999999

In [30]:
# NDCG@5 на тестовой выборке

y_pred_test = model_cb.predict(X_test)
ndcg_score([y_test], [y_pred_test], k=5)

0.6854018932919382

Создаим простую модель XGBoost

In [34]:
import xgboost as xgb

xgb.__version__

'2.0.3'

In [79]:
df_train = df[df['query_id'] <= 22495]
df_test = df[df['query_id'] > 22495]

In [80]:
X_train = df_train.drop(columns=['rank', 'query_id'])
y_train = df_train['rank']
queries_train = df_train['query_id']

In [81]:
X_test = df_test.drop(columns=['rank', 'query_id'])
y_test = df_test['rank']
queries_test = df_test['query_id']


In [82]:
model_xgb = xgb.XGBRanker(
    max_depth =  10,
    objective = 'rank:ndcg',
    eval_metric=["ndcg"],
    seed = 42,
    device='gpu'
)
model_xgb.fit(
    X_train,
    y_train,
    qid=queries_train,
    verbose=False
)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device='gpu',
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=['ndcg'], feature_types=None, gamma=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [85]:
# NDCG@5 на тренировочной выборке

y_pred_tain = model_xgb.predict(X_train)
ndcg_score([y_train], [y_pred_tain], k=5)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [23:09:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


0.9999999999999999

In [86]:
# NDCG@5 на тестовой выборке

y_pred_test = model_xgb.predict(X_test)
ndcg_score([y_test], [y_pred_test])

0.9268140700396167

В этой работе я сравнил результаты библиотек CatBoost, XGBoost.  
Метрики XGBoost получились лучше, чем у CatBoost.  
Предположительно это связано с переобучением модели.  

|     | CatBoost | XGB |
| --- | --- | --- |
| train | 0.99 | 0.69 |
|test | 0.99 | 0.93 |


В качестве улучшений проделанной работы можно рассмотреть работу в следующих направлениях:

- отбор признаков датасета,
- генерация новых признаков,
- собрать ансамбль ранжирующих моделей.